# GST on Rigetti Backends with PyGSTi
The purpose of this notebook is to use the data collected from a GST experiment that has been read into the `rigetti_first_run` directory and perform GST on it. 

This code was mostly taken from the tutorial notebook [UnitaryFundDemoNotebook1.ipynb
](https://zenodo.org/record/5715199#.Yr_TOUjMLJF). This notebook also exhibits pyGSTi's full html report functionality, but there is a conflict with the latest version of jinja2 which breaks this.

In [1]:
#PyGSTi tools
import pygsti
from pygsti.data.dataset import DataSet

#Mitiq and pyGSTi seem to require competing versions of numpy,
#so output is pickled and passed to a different notebook
import pickle

#Rigetti tools
from forest.benchmarking.operator_tools.superoperator_transformations import pauli_liouville2superop
from pyquil import Program

/home/ben/.local/lib/python3.10/site-packages/pygsti/baseobjs/opcalc/__init__.py:31: UserWarning: 
An optimized Cython-based implementation of `pygsti.baseobjs.opcalc` is available as
an extension, but couldn't be imported. This might happen if the
extension has not been built. `pip install cython`, then reinstall
pyGSTi to build Cython extensions. Alternatively, setting the
environment variable `PYGSTI_NO_CYTHON_WARNING` will suppress this
message.

  _warnings.warn(warn_msg)


## Read data from experiment
This data was written by the `get_gst_data.ipynb` file.

In [2]:
data = pygsti.io.read_data_from_dir('experiment_data/rigetti_first_run')

## Run GST on the data

In [3]:
gst_protocol = pygsti.protocols.StandardGST('TP,CPTP,Target')
results = gst_protocol.run(data)

-- Std Practice:  Iter 1 of 3  (TP) --: 
  --- Iterative GST: [##################################################] 100.0%  550 circuits ---
  Iterative GST Total Time: 1.5s
-- Std Practice:  Iter 2 of 3  (CPTP) --: 


  --- Iterative GST: [##################################################] 100.0%  550 circuits ---
  Iterative GST Total Time: 16.6s
-- Std Practice:  Iter 3 of 3  (Target) --: 


## Create GST report

In [4]:
report = pygsti.report.construct_standard_report(
    results, title="GST Overview Tutorial Example Report")

Running idle tomography
Computing switchable properties


## Constrain to TPCP and gauge-optimize
The estimates_final_model is an `ExplicitOpModel` type, which carries the preps, measurements, and operations as Pauli-transfer matrices. The documentation refers to this as the 'pauli-product' basis. The `forest_benchmarking` tools can be used to convert these into superoperators, which is the form mitiq uses to describe quantum channels.

In [13]:
cptp_estimate = results.estimates['CPTP']
cptp_estimate_final_model = cptp_estimate.models['stdgaugeopt']
print(cptp_estimate_final_model)

rho0 = TPState with dimension 4
 0.71 0.02 0.03 0.69


Mdefault = TPPOVM with effect vectors:
0: FullPOVMEffect with dimension 4
 0.81-0.01-0.03 0.61

1: ComplementPOVMEffect with dimension 4
 0.61 0.01 0.03-0.61



Gxpi2:0 = 
FullTPOp with shape (4, 4)
 1.00   0   0   0
   0 0.99 0.03-0.03
   0-0.03 0.01-0.98
   0-0.03 0.98 0.01


Gzpi2:0 = 
FullTPOp with shape (4, 4)
 1.00   0   0   0
   0   0-1.00-0.03
   0 1.00   0 0.03
   0-0.03-0.03 1.00






## Gauge optimization

In [14]:
target_cptp_estimate = results.estimates['Target']
target_mdl = target_cptp_estimate.models['stdgaugeopt']
print(target_mdl)


rho0 = TPState with dimension 4
 0.71   0   0 0.71


Mdefault = TPPOVM with effect vectors:
0: FullPOVMEffect with dimension 4
 0.71   0   0 0.71

1: ComplementPOVMEffect with dimension 4
 0.71   0   0-0.71



Gxpi2:0 = 
FullTPOp with shape (4, 4)
 1.00   0   0   0
   0 1.00   0   0
   0   0   0-1.00
   0   0 1.00   0


Gzpi2:0 = 
FullTPOp with shape (4, 4)
 1.00   0   0   0
   0   0-1.00   0
   0 1.00   0   0
   0   0   0 1.00






In [31]:
# from pygsti.models.gaugegroup import FullGaugeGroup                                                          

gauge_optimized_mdl = pygsti.algorithms.gaugeopt_to_target(cptp_estimate_final_model, 
                                                           target_mdl,
                                                           # gauge_group=FullGaugeGroup(1),
                                                           item_weights={'Gz': 1.0, 'gates': 0.0, "spam": 0.0},
                                                           verbosity=3,
                                                           tol=10^9,
)
print(gauge_optimized_mdl)

  --- Gauge Optimization (ls method, <class 'pygsti.models.gaugegroup.TPGaugeGroup'>) ---
  Least squares message = Sum of squares is at most 3
Gauge optimization completed in 0.0107453s.
rho0 = TPState with dimension 4
 0.71 0.02 0.03 0.69


Mdefault = TPPOVM with effect vectors:
0: FullPOVMEffect with dimension 4
 0.81-0.01-0.03 0.61

1: ComplementPOVMEffect with dimension 4
 0.61 0.01 0.03-0.61



Gxpi2:0 = 
FullTPOp with shape (4, 4)
 1.00   0   0   0
   0 0.99 0.03-0.03
   0-0.03 0.01-0.98
   0-0.03 0.98 0.01


Gzpi2:0 = 
FullTPOp with shape (4, 4)
 1.00   0   0   0
   0   0-1.00-0.03
   0 1.00   0 0.03
   0-0.03-0.03 1.00






In [26]:
from pygsti.modelpacks import smq1Q_XYZI
initial_model = smq1Q_XYZI.target_model()
# Change all of initial_model's operations to having a CPTP-constrained parameterization.
initial_model.set_all_parameterizations("CPTP")
# Replace the CPTP-constrained Z-gate with an always-perfect (0 parameter, "static") Z-gate.
gate_matrix = initial_model[('Gzpi2',0)].to_dense()

In [30]:

initial_model[('Gzpi2',0)] = pygsti.modelmembers.operations.staticarbitraryop.StaticArbitraryOp(gate_matrix)
# Run GST (on existing gst_data)
results = pygsti.protocols.GST(initial_model, gaugeopt_suite=None).run(results)

AttributeError: 'ModelEstimateResults' object has no attribute 'edesign'

In [69]:
import numpy as np

before = cptp_estimate_final_model.operations[('Gzpi2',0)]
after = gauge_optimized_mdl.operations[('Gzpi2',0)]

np.allclose(before, after)

True

## Save data

In [ ]:
# Convert operator transfer matrix to superoperator
superop_rep_x = pauli_liouville2superop(cptp_estimate_final_model.operations[('Gxpi2',0)])
superop_rep_z = pauli_liouville2superop(cptp_estimate_final_model.operations[('Gzpi2',0)])

# Pickle them to pass them to Mitiq_representation notebook to be used for PEC
with open("experiment_data/XZ.superop", "wb") as f:
    pickle.dump([superop_rep_x,superop_rep_z],f)